# Table of Contents :
* [1. Problem Introduction](#section1)
* [2. Data preprocessing](#section2)
* [3. Modelling](#section3)
  




<a id="section1"></a>
# Introduction :
<p style="font-size:15px; font-family:verdana; line-height: 1.7em">We are given a dataset about various books category and their picture. 
We will classification book into right categories</p>

In [41]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [42]:
images = []
targets =[]
flatten_img = []

In [43]:
DATADIR = './images'
CATS = ["Sách_Giáo_Khoa_Cấp_1","Sách_Giáo_Khoa_Cấp_2","Sách_Giáo_Khoa_Cấp_3"]
for cat in CATS:
    class_num = CATS.index(cat)
    path = os.path.join(DATADIR, cat)
    #print(path)
    for item in os.listdir(path):
        img =cv2.imread(os.path.join(path, item), 1)
        #print(img.shape)
        # plt.imshow(img)
        # plt.show()
        #print(item)
        flatten_img.append(img.flatten())
        images.append(img)
        targets.append(class_num)

In [44]:
images_np = np.array(images)
targets_np = np.array(targets)
flatten_img_np =  np.array(flatten_img)
print(images_np.shape)
print(targets_np.shape)
print(flatten_img_np.shape)

(100, 280, 200, 3)
(100,)
(100, 168000)


In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(flatten_img_np, targets, test_size=0.2, random_state=42)

In [46]:
y_train

[1,
 2,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 1]

In [47]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV,cross_val_score

param_grid = [
    {'C':[1,10,100,1000], 'kernel':["linear"]},
    {'C':[1,10,100,1000], 'gamma':[0.001, 0.0001] ,'kernel':["rbf"]}
]
svc = svm.SVC()
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train,y_train)


GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             scoring='accuracy')

In [50]:
results = grid_search.cv_results_
for mean_score, params in zip(results["mean_test_score"], results["params"]):
    print(mean_score, params)

0.5 {'C': 1, 'kernel': 'linear'}
0.5 {'C': 10, 'kernel': 'linear'}
0.5 {'C': 100, 'kernel': 'linear'}
0.5 {'C': 1000, 'kernel': 'linear'}
0.4375 {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.4375 {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.4375 {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.4375 {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.4375 {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.4375 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.4375 {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.4375 {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}


In [52]:
from sklearn.metrics import accuracy_score

svc_best =  grid_search.best_estimator_

cv_scrore = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='accuracy')
print(cv_scrore)

[0.5625 0.625  0.4375 0.4375 0.4375]
